In [35]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [36]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import ast
import json

In [55]:

# Load your data (replace with your file path)
#data= pd.read_parquet('your_landmark_data.parquet')

df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks_lh_rh")


In [ ]:
print(df_from_parquet)

In [78]:
unique_labels = df_from_parquet['Label'].unique()
label_id_mapping = {label: idx for idx, label in enumerate(unique_labels)}
# Step 3: Map the Label to Label_id in the DataFrame
df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
label_info = [{label, label_id} for label, label_id in label_id_mapping.items()]

In [ ]:
df_from_parquet

In [88]:
combined_set = {tuple(s) for s in df_from_parquet['Keypoints']}

In [ ]:
combined_set

In [93]:
arr=df_from_parquet[['Label_id','Sample_Number','Frame_Number','Keypoints']].to_numpy()

In [75]:
arr=df_from_parquet[['Label','Sample_Number','Frame_Number','Keypoints']].values.tolist()

In [ ]:
arr

In [ ]:
label_id_mapping

In [43]:
# Step 3: Map the Label to Label_id in the DataFrame
df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
label_info = [{label, label_id} for label, label_id in label_id_mapping.items()]

In [ ]:
label_info

In [47]:

# Step 1: Ensure Keypoints are converted to lists if they are not already
df_from_parquet['Keypoints'] = df_from_parquet['Keypoints'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Step 2: Group by the necessary columns and aggregate
grouped = df_from_parquet.groupby(['Label','Label_id', 'sample_type', 'Sample_Number']).apply(
    lambda x: x[['Frame_Number', 'Keypoints']].to_dict('records')
).reset_index(name='frames_keypoints')

# Step 3: Convert the grouped result to a JSON-like structure
grouped_json = grouped.to_dict(orient='records')

# Step 5: Convert the result to a JSON string
json_output = json.dumps(grouped_json, indent=4)


In [ ]:
# Step 5: Ensure grouped_json is treated as an array
json_output_array = [grouped_json]  # or simply treat it as is
json_output = json.dumps(json_output_array, indent=4)

# If grouped_json is already a list, you can omit wrapping it:
json_output = json.dumps(grouped_json, indent=4)

# Printing or using the json output
print(json_output)

In [ ]:
# Step to remove names and keep only numbers in the grouped_json structure
def remove_names_and_keep_numbers(entry):
    return {
        'Label_id': entry['Label_id'],
        'Sample_Number': entry['Sample_Number'],
        'frames_keypoints': [
            {'Frame_Number': frame['Frame_Number'], 'Keypoints': frame['Keypoints']}
            for frame in entry['frames_keypoints']
        ]
    }

# Applying this function to the grouped_json
numeric_grouped_json = [remove_names_and_keep_numbers(item) for item in grouped_json]

# Convert to JSON string
json_output = json.dumps(numeric_grouped_json, indent=4)

# Printing or using the output
print(json_output)


In [ ]:
print(grouped_json[0])


In [ ]:
label_id_mapping

In [ ]:
data[0]

In [5]:
data = json.loads(json_output)
#print(data)

In [ ]:
np.array(array_output).shape

In [4]:
import pandas as pd
import json

# Read from a parquet file

# Proceed with mapping and processing as before
unique_labels = df_from_parquet['Label'].unique()
label_id_mapping = {label: idx for idx, label in enumerate(unique_labels)}

df_from_parquet['Label_id'] = df_from_parquet['Label'].map(label_id_mapping)
df_from_parquet['Keypoints'] = df_from_parquet['Keypoints'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

# Ensure this is a DataFrame before passing it
if isinstance(df_from_parquet, pd.DataFrame):
    grouped = df_from_parquet.groupby(['Label', 'Label_id', 'sample_type', 'Sample_Number']).apply(
        lambda x: x[['Frame_Number', 'Keypoints']].to_dict('records')
    ).reset_index(name='frames_keypoints')
    
    grouped_json = grouped.to_dict(orient='records')
    json_output = json.dumps(grouped_json, indent=4)

    # Optionally save to a file
    with open('output.json', 'w') as json_file:
        json_file.write(json_output)
else:
    print("Data is not a DataFrame.")


In [ ]:
data = json.loads(json_output)
print(data[0])

In [25]:
import re
import numpy as np

def extract_keypoints(df):
    keypoints = []
    labels = []
    
    for _, row in df.iterrows():
        keypoint_str = row[1]['frames_keypoints'][0]['Keypoints']
        
        if isinstance(keypoint_str, str):
            print("Is Str")
            # Remove curly braces and split the string by commas
            keypoint_str = keypoint_str.replace('{', '').replace('}', '')
            try:
                print("Is not Str")
                # Convert the string into a list of floats
                keypoint_list = [float(val) for val in keypoint_str.split(',')]
                keypoint_str = keypoint_list.replace('{', '').replace('}', '')
            except ValueError as e:
                print(f"Error converting keypoints: {keypoint_str} -> {e}")
                continue  # Skip problematic rows if conversion fails
        else:
            keypoint_list = keypoint_str  # If not a string, assume it's already correct
        
        keypoints.append(keypoint_list)
        labels.append(row['Label'])  # Assuming label is numeric
    
    return np.array(keypoints), np.array(labels)


In [ ]:
data.iterrows()

In [ ]:
data[0]
#['frames_keypoints'][0]['Keypoints']

In [ ]:
X, y = extract_keypoints(data)


In [ ]:

# Check keypoints data and ensure it's numeric
print("Keypoints shape: ", X.shape)
print("Sample keypoints data: ", X[0])



In [ ]:
X = X.astype(np.float32)  # Convert keypoints to float32
X = X / np.max(X)  # Normalize between 0 and 1
